# 2th Technical Meeting - Unet Tutorial

This material is part of 2th Technical Meeting of NexGenMap project  [(event website)](https://sites.google.com/view/nextgenmap2reuniao/página-inicial).

The NexGenMap is a project that aims establish the next generation of land use and land cover mapping and monitoring tools, using high-res images (i.e. Planet data), cloud-computing platforms (i.e. Google Earth Engine and Google Cloud Computing) and machine learning and deep-learning techniques (e.g. Random Forest, U-Net and LSTM).

The main goal of this tutorial is run a semantic segmentation deep-learning approach, through U-net, to identify cattle drinking fountains using monthly Planet mosaics of NexGenMap. It's organized:
* Data and Libraries
* Data Exploration
* Standardizing/Normalization
* Training dataset
* U-net model (train/validation/prediction)

## Data and Libraries

In [1]:
import nexutil

In [2]:
#file_url = 'https://www.lapig.iesa.ufg.br/lapig/nextgenmap-data/tutorial-sessions/02-2th_Technical_Meeting_U-net/input_images.zip'
#filename = 'input_images.zip'

#nexutil.download_extract_zipfile(file_url, filename)

In [3]:
#nexutil.image_info('input_images/SD-22-Z-C-alfa_2-1.tif')

In [4]:
#nexutil.vis_image('input_images/SD-22-Z-C-alfa_2-1.tif', bands=[1,2,3])

In [5]:
#nexutil.vis_image('input_images/SD-22-Z-C-alfa_2-1.tif', bands=[5,6,7])

In [6]:
#nexutil.vis_image('input_images/SD-22-Z-C-alfa_2-1.tif', bands=[8])

In [7]:
file_url = 'https://github.com/NexGenMap/dl-semantic-segmentation/archive/master.zip'
filename = 'master.zip'

#nexutil.download_extract_zipfile(file_url, filename)
nexutil.import_repository('dl-semantic-segmentation-master/src/')

In [8]:
#%%bash 
#pip3 install --user psutil

In [9]:
import run

In [10]:
print(run.params)

{'seed': 1989, 'learning_rate': 1e-05, 'l2_regularizer': 0.5, 'batch_size': 32, 'nodata_value': -10, 'n_input_bands': 9, 'chip_size': 100, 'n_classes': 2, 'tensorboard_maxoutput': 2, 'num_epochs': 200, 'pad_size': 93, 'dropout_rate': 0.5, 'model_dir': 'logs_bebedouros'}


In [11]:
img_input_path = 'input_images/SD-22-Z-C-alfa_2-1.tif'
img_input_nodata = -10
run.params['num_epochs'] = 1
#run.train(img_input_path, run.params)

In [12]:
run.evaluate(img_input_path, run.params)

Reading from cache input_images/SD-22-Z-C-alfa_2-1.npz...
Train samples:  124
Test samples:  32
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_model_dir': 'logs_bebedouros', '_save_summary_steps': 100, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_evaluation_master': '', '_task_id': 0, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd538a7b0f0>, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_session_config': None, '_save_checkpoints_steps': None, '_service': None, '_master': ''}
Evaluating results for image input_images/SD-22-Z-C-alfa_2-1.tif...
Saving images in results/train...
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph wa